# Method and Result 

Using an appropriately tuned deep feed-forward neural network to train the data ($2000$ training examples, with each example consisting of a time series/trajectory for the $x$-coordinate of 102 steps), we found that accuracy on the test data ($1000$ test examples, with each example consisting of a trajectory for the $x$-coordinate of 102 steps) is around $73 \%$, which could be improved by using, for instance, more training data, including the trajectory for the $y$-coordinate in the training data set, or a more sophisticated neural network architecture. 

In [412]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
#plt.style.use('fivethirtyeight')
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from keras.optimizers import SGD
import math
from sklearn.metrics import mean_squared_error

In [413]:
# First, we get the data
dataset0 = pd.read_csv("/Users/soonhoe/Desktop/AAA Python/trajx_m0.csv",header=None)
dataset1 = pd.read_csv("/Users/soonhoe/Desktop/AAA Python/trajx_m1.csv",header=None)
dataset2 = pd.read_csv("/Users/soonhoe/Desktop/AAA Python/trajx_m2.csv",header=None)

li = []
li.append(dataset0)
li.append(dataset1)
li.append(dataset2)

dataset = np.array(pd.concat(li, axis=0, ignore_index=True))
np.random.shuffle(dataset)
dataset = pd.DataFrame(dataset)

np.savetxt('all_traj.csv', np.c_[dataset], delimiter=',') 

dataset.head(10)
row=np.shape(dataset)[0]
col=np.shape(dataset)[1]
print(row); print(col)

3000
102


In [414]:
# Checking for missing values
num_train = int(row*2/3)
training_set = dataset[:num_train].iloc[:,0:col].values
test_set = dataset[num_train:].iloc[:,0:col].values
print(np.shape(training_set)); print(np.shape(test_set))

(2000, 102)
(1000, 102)


In [415]:
training_df = pd.DataFrame(training_set)
training_df.head(10)

#pd.plotting.scatter_matrix(training_df, c=training_set['750'], figsize=(15,15), marker='o', s=60)
#plt.show()

,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,100,101
0,0.0,0.014233,0.018073,0.033469,0.008009,-0.010139,0.000483,-0.029256,-0.042169,-0.068667,...,-0.052008,-0.036385,-0.033684,-0.009525,-0.027281,-0.030483,-0.041839,-0.054744,-0.067227,0.0
1,0.0,0.013381,0.017872,0.011803,0.017269,0.000612,0.003238,0.008392,0.011141,-0.007943,...,-0.458650,-0.490972,-0.512801,-0.512246,-0.496050,-0.502954,-0.501706,-0.487807,-0.477138,1.0
2,0.0,-0.037596,0.016685,0.025684,0.043444,0.051707,0.053185,0.048018,0.038871,0.024177,...,0.092414,0.091177,0.098155,0.084328,0.077293,0.066811,0.082621,0.073460,0.120050,0.0
3,0.0,-0.007297,0.007790,0.002695,0.000978,-0.007581,0.009402,0.024470,0.012727,0.020929,...,0.146325,0.169303,0.194519,0.188065,0.183606,0.189446,0.197405,0.190020,0.205637,0.0
4,0.0,-0.000030,-0.010173,0.011853,-0.001674,0.001349,-0.007944,-0.026456,-0.034731,-0.056660,...,-0.539661,-0.547715,-0.534951,-0.519082,-0.527979,-0.564180,-0.557321,-0.560317,-0.576512,1.0
5,0.0,0.000535,0.003537,0.023761,0.029217,0.046976,0.038772,0.032941,0.011174,-0.005505,...,-0.027107,-0.011182,0.002101,0.007328,0.020003,0.034761,0.026486,0.013341,-0.005342,0.0
6,0.0,-0.005443,0.022262,-0.020492,-0.023009,-0.021155,-0.021481,-0.002577,-0.005792,0.007509,...,-0.352719,-0.368043,-0.365271,-0.366456,-0.378613,-0.395740,-0.395808,-0.406339,-0.388433,2.0
7,0.0,-0.002420,0.010192,-0.020471,-0.011108,0.000158,0.004495,-0.012705,-0.009855,-0.001137,...,-0.018343,-0.030837,-0.056446,-0.041564,-0.009953,-0.008850,-0.008732,-0.024432,-0.026219,0.0
8,0.0,-0.027301,-0.011466,-0.030466,-0.062386,-0.061362,-0.067246,-0.094593,-0.107024,-0.120600,...,-0.391666,-0.427885,-0.447803,-0.444593,-0.449535,-0.423673,-0.428646,-0.417457,-0.426954,2.0
9,0.0,0.006047,-0.008136,-0.005089,0.018195,-0.000838,-0.009116,-0.056756,-0.074604,-0.064575,...,-0.589496,-0.620288,-0.621947,-0.637385,-0.641903,-0.627130,-0.639971,-0.626084,-0.635690,2.0


In [416]:
from __future__ import print_function

import keras
#from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

#hyperparameters
batch_size = 500
num_classes = 3
epochs = 100

# the data, split between train and test sets
x_train=training_set[:,:col-1]
y_train=training_set[:,col-1:]
x_test=test_set[:,:col-1]
y_test=test_set[:,col-1:]

x_train = x_train.reshape(num_train, col-1)
x_test = x_test.reshape(row-num_train, col-1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
#x_train /= 255
#x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print(np.shape(x_train)); print(np.shape(x_test))
print(np.shape(y_train)); print(np.shape(y_test))

2000 train samples
1000 test samples
(2000, 101)
(1000, 101)
(2000, 1)
(1000, 1)


In [395]:
# convert class vectors to class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Dense(256*2, activation='relu', input_shape=(col-1,)))
model.add(Dropout(0.2))
model.add(Dense(256*2, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
classes = model.predict(x_test, batch_size=batch_size)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_238 (Dense)            (None, 512)               52224     
_________________________________________________________________
dropout_166 (Dropout)        (None, 512)               0         
_________________________________________________________________
dense_239 (Dense)            (None, 512)               262656    
_________________________________________________________________
dropout_167 (Dropout)        (None, 512)               0         
_________________________________________________________________
dense_240 (Dense)            (None, 3)                 1539      
Total params: 316,419
Trainable params: 316,419
Non-trainable params: 0
_________________________________________________________________
Train on 2000 samples, validate on 1000 samples
Epoch 1/100
2000/2000 [==============================] - 5s 3ms/step - loss: 0.9790 - ac

In [396]:
print(classes)
print(np.shape(classes)) #probability distribution for each of the 3 models

[[7.5622219e-01 9.6915674e-04 2.4280863e-01]
 [7.5979722e-01 1.1964265e-03 2.3900643e-01]
 [6.1479761e-05 6.1850911e-01 3.8142937e-01]
 ...
 [6.8936497e-01 2.1743931e-02 2.8889111e-01]
 [9.6748382e-01 6.1194744e-04 3.1904247e-02]
 [2.6304166e-05 6.0386640e-01 3.9610732e-01]]
(1000, 3)
